In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import cv
import matplotlib.pyplot as plt

In [2]:
from scipy.stats import  kstest  

In [3]:
from catboost import CatBoostClassifier

In [4]:
from tqdm import tqdm

In [5]:
from sklearn.model_selection import cross_val_score , StratifiedKFold
from sklearn.metrics import roc_auc_score

In [6]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

Применение техники Adversarial validation для проверки равенства распределений признаков в train и test.

Датасеты train и test содиняются в один набор и добавляется целевая колонка, индикатор того в каком наборе данных объект находится. 

* 0 - train
* 1 - test

На этом наборе данных обучается модель CatBoost и оценивается на кросс-валидации. Оценка ROC-AUC. Если в среднем ROC-AUC равен примерно 0.5 - модель затрудняется распознать выборку, в которой содержится объект. Наборы данных составлены правильно.


Загрузка данных

In [7]:
train = pd.read_csv('train.csv' , sep = ',').drop(['price','Property_ID','locality' ] , axis = 1)
test = pd.read_csv('test.csv' , sep = ',').drop(['price','Property_ID','locality' ] , axis = 1)

In [8]:
cat_cols = train.columns[ (train.columns != 'security_deposit') &     (train.columns != 'area' ) ]

In [8]:
train[ cat_cols ] =  train[ cat_cols ].astype("category")
test[cat_cols ] =  test[ cat_cols].astype("category")

In [62]:
 train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134683 entries, 0 to 134682
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   room              134683 non-null  category
 1   layout_type       134683 non-null  category
 2   property_type     134683 non-null  category
 3   area              134683 non-null  int64   
 4   furnish_type      134683 non-null  category
 5   bathroom          134683 non-null  category
 6   city              134683 non-null  category
 7   parking_spaces    134683 non-null  category
 8   floor             134683 non-null  category
 9   pet_friendly      134683 non-null  category
 10  power_backup      134683 non-null  category
 11  washing_machine   134683 non-null  category
 12  air_conditioner   134683 non-null  category
 13  geyser/solar      134683 non-null  category
 14  security_deposit  134683 non-null  int64   
 15  CCTV/security     134683 non-null  category
 16  li

In [63]:
# индикатор - в какой выборке находится объект
train_label = pd.Series([0] * train.shape[0] + [1] * test.shape[0] )

In [64]:
# соединенные train и test
all_obj = pd.concat( [train , test] ,  axis=0, ignore_index=True)

In [65]:
# перемешанный датасет
all_obj_shuffled = all_obj.sample(frac=1)

In [66]:
# перемешанный целевой столбец
train_label_shuffled =  train_label[ all_obj_shuffled.index ]

In [81]:
# инициализация модели CatBoost
params = {
    'iterations': 500,
    'eval_metric': 'AUC',
    'cat_features'  : list(cat_cols) , 
    'verbose' : False 
}
model = CatBoostClassifier(**params)

In [85]:
# оценка на кросс-валидации
CV_str = StratifiedKFold(n_splits=5,  shuffle=True, random_state=42 )

cv_score_params = {
    'estimator': model, 
    'X': all_obj_shuffled ,
    'y': train_label_shuffled ,
    'cv': CV_str,
    'scoring': 'roc_auc'
}
 
cv_score_auc = cross_val_score( **cv_score_params )

In [86]:
cv_score_auc

array([0.50149253, 0.50426245, 0.49779192, 0.49981842, 0.49809588])

Оценки на кросс-валидации примерно равны 0.5 . Наборы данных разделены правильно.

 Тоже самое можно сделать при помощи тестов на равенство распределений.

In [90]:
# тест Колмогорова-Смирнова на равенство распределений признака из train и test
for I in train.columns:
    print( 'Проверка распределения для признака ', I)
    stat , pv =  kstest(rvs = train[ I ] , 
                        cdf = test[ I ])
    if pv < 0.05:
        print('Pval = ',pv  ,' Распределения неодинаковы')
    else:
        print('Pval = ',pv  ,' Распределение одинаковы')
 

Проверка распределения для признака  room
Pval =  0.9732287557428543  Распределение одинаковы
Проверка распределения для признака  layout_type
Pval =  0.7035022318647677  Распределение одинаковы
Проверка распределения для признака  property_type
Pval =  0.9999986150797623  Распределение одинаковы
Проверка распределения для признака  area
Pval =  0.2247106329342453  Распределение одинаковы
Проверка распределения для признака  furnish_type
Pval =  0.9999999999999953  Распределение одинаковы
Проверка распределения для признака  bathroom
Pval =  0.9999977922921935  Распределение одинаковы
Проверка распределения для признака  city
Pval =  0.15070196211760278  Распределение одинаковы
Проверка распределения для признака  parking_spaces
Pval =  0.9994024860702254  Распределение одинаковы
Проверка распределения для признака  floor
Pval =  0.9983449844424205  Распределение одинаковы
Проверка распределения для признака  pet_friendly
Pval =  0.7983612365823595  Распределение одинаковы
Проверка рас

Проверка парных взаимодействий категориальных переменных.

Проверка равенства распределений в train и test.


In [138]:
two_way_inter_train = pd.DataFrame()
two_way_inter_test = pd.DataFrame()

trn_inter_cols  = []
tst_inter_cols = []

In [139]:
for i in range( len(cat_cols )-1):
    for k in range( i+1,  len(cat_cols )):
        print('========================================================')
        s = pd.crosstab(train[ cat_cols[i] ] , train[ cat_cols[k] ] ).values
        print(pd.crosstab(train[ cat_cols[i] ] , train[ cat_cols[k] ] ))
        if ( s[s==0].size == 0 ):
            print(cat_cols[i] , ' x ', cat_cols[k])
            trn_inter = train[cat_cols[i] ].astype('str')+' x ' + train[ cat_cols[k]].astype('str')
            tst_inter = test[cat_cols[i] ].astype('str')+' x ' + test[cat_cols[k]].astype('str')
            trn_inter_cols.append( cat_cols[i] + ' x '+ cat_cols[k] )
            tst_inter_cols.append( cat_cols[i] + ' x '+ cat_cols[k])
            stat , pv =  kstest(rvs = trn_inter , 
                        cdf = tst_inter)
            if pv < 0.05:
                print('Pval = ',pv  ,' Распределения неодинаковы')
            else:
                two_way_inter_train = pd.concat([two_way_inter_train ,trn_inter ] , axis=1)
                two_way_inter_test = pd.concat([two_way_inter_test ,tst_inter ] , axis=1)
                print('Pval = ',pv  ,' Распределения одинаковы')
                
two_way_inter_train.columns =  trn_inter_cols 
two_way_inter_test.columns =tst_inter_cols

layout_type    BHK    RK
room                    
1            38204  9704
2            38418  4817
3            31709  4416
4             5555   910
5              798   152
room  x  layout_type
Pval =  0.9732287557428543  Распределения одинаковы
property_type  Apartment  Independent Floor  Independent House  Penthouse  \
room                                                                        
1                  27894               9802               5497        244   
2                  29528               8115               3842        112   
3                  24582               6572               3288        101   
4                   4225               1188                666         31   
5                    590                173                115          4   

property_type  Studio Apartment  Villa  
room                                    
1                          3158   1313  
2                          1149    489  
3                          1127    455  
4      

Pval =  0.5913998640376391  Распределения одинаковы
furnish_type  Furnished  Semi-Furnished  Unfurnished
layout_type                                         
BHK               25390           56027        33267
RK                 4249            7619         8131
layout_type  x  furnish_type
Pval =  0.27314257406214215  Распределения одинаковы
bathroom         1      2      3     4     5
layout_type                                 
BHK          27021  63386  15913  6845  1519
RK            7539   8393   2544  1260   263
layout_type  x  bathroom
Pval =  0.6065102144926866  Распределения одинаковы
city         Ahmedabad  Bangalore  Chennai  Delhi  Hyderabad  Kolkata  Mumbai  \
layout_type                                                                     
BHK              11597      14109     5189  19436       6201     5761   39471   
RK                1379       1983      848   3390       1133     1034    7439   

city          Pune  
layout_type         
BHK          12920  
RK       

Pval =  0.9961724372977396  Распределения одинаковы
pet_friendly           0      1
property_type                  
Apartment          41218  45601
Independent Floor  12348  13502
Independent House   6181   7227
Penthouse            198    294
Studio Apartment    2599   3124
Villa               1080   1311
property_type  x  pet_friendly
Pval =  0.9528138401243438  Распределения одинаковы
power_backup           0      1
property_type                  
Apartment          57284  29535
Independent Floor  17267   8583
Independent House   8913   4495
Penthouse            322    170
Studio Apartment    3903   1820
Villa               1599    792
property_type  x  power_backup
Pval =  0.9776456183961191  Распределения одинаковы
washing_machine        0      1
property_type                  
Apartment          43891  42928
Independent Floor  14078  11772
Independent House   7433   5975
Penthouse            290    202
Studio Apartment    3782   1941
Villa               1563    828
property_type 

Pval =  0.9999999999999953  Распределения одинаковы
lift                0      1
furnish_type                
Furnished       12081  17558
Semi-Furnished  24357  39289
Unfurnished     17994  23404
furnish_type  x  lift
Pval =  0.9999797348311348  Распределения одинаковы
neighbourhood     0   100   200   300   400   500   600   700   800   900  \
furnish_type                                                                
Furnished       439  1238   551   571   583   588   571   665   664   686   
Semi-Furnished  852  2547  1158  1281  1323  1172  1269  1420  1414  1566   
Unfurnished     625  1427   548   704   752   744   780   877   828   908   

neighbourhood   ...  3100  3200  3300  3400  3500  3600  3700  3800  3900  \
furnish_type    ...                                                         
Furnished       ...   631   626   713   727   773   781   684   583  1232   
Semi-Furnished  ...  1432  1424  1418  1525  1550  1590  1340  1163  2337   
Unfurnished     ...   953   960   9

Pval =  0.10065284741278069  Распределения одинаковы
pet_friendly      0      1
city                      
Ahmedabad      6277   6699
Bangalore      7679   8413
Chennai        2892   3145
Delhi         10725  12101
Hyderabad      3391   3943
Kolkata        3144   3651
Mumbai        22011  24899
Pune           7505   8208
city  x  pet_friendly
Pval =  0.15070196211760278  Распределения одинаковы
power_backup      0      1
city                      
Ahmedabad      8350   4626
Bangalore     10556   5536
Chennai        3886   2151
Delhi         15073   7753
Hyderabad      4793   2541
Kolkata        4514   2281
Mumbai        31345  15565
Pune          10771   4942
city  x  power_backup
Pval =  0.15070196211760278  Распределения одинаковы
washing_machine      0      1
city                         
Ahmedabad         5034   7942
Bangalore         6960   9132
Chennai           3004   3033
Delhi            12036  10790
Hyderabad         4058   3276
Kolkata           3839   2956
Mumbai           

Pval =  0.9154231101891309  Распределения одинаковы
power_backup     0     1
floor                   
0             3073  1674
1             4613  2474
2             8355  4268
3             6044  2820
4             5571  2724
5             4614  2345
6             3901  1956
7             3276  1659
8             3546  1711
9             3877  1838
10            4203  2116
11            4188  2041
12            3953  1972
13            3887  1904
14            3446  1715
15            4549  2471
16            4144  2312
17            5332  2770
18            4038  2083
19            4678  2542
floor  x  power_backup
Pval =  0.6763566613825955  Распределения одинаковы
washing_machine     0     1
floor                      
0                2434  2313
1                3812  3275
2                7588  5035
3                5151  3713
4                4543  3752
5                3622  3337
6                2925  2932
7                2406  2529
8                2738  2519
9              

Pval =  0.7983612365823595  Распределения одинаковы
geyser/solar      0      1
pet_friendly              
0             35186  28438
1             40218  30841
pet_friendly  x  geyser/solar
Pval =  0.7983612365823595  Распределения одинаковы
CCTV/security      0      1
pet_friendly               
0              27800  35824
1              31213  39846
pet_friendly  x  CCTV/security
Pval =  0.7983612365823595  Распределения одинаковы
lift              0      1
pet_friendly              
0             25856  37768
1             28576  42483
pet_friendly  x  lift
Pval =  0.6501015353089449  Распределения одинаковы
neighbourhood     0   100   200   300   400   500   600   700   800   900  \
pet_friendly                                                                
0               860  2500  1065  1219  1279  1171  1227  1320  1353  1456   
1              1056  2712  1192  1337  1379  1333  1393  1642  1553  1704   

neighbourhood  ...  3100  3200  3300  3400  3500  3600  3700  3800  3900

Pval =  0.7465161534538465  Распределения одинаковы
lift               0      1
CCTV/security              
0              23837  35176
1              30595  45075
CCTV/security  x  lift
Pval =  1.0  Распределения одинаковы
neighbourhood     0   100   200   300   400   500   600   700   800   900  \
CCTV/security                                                               
0               826  2415  1077  1171  1180  1088  1176  1315  1301  1437   
1              1090  2797  1180  1385  1478  1416  1444  1647  1605  1723   

neighbourhood  ...  3100  3200  3300  3400  3500  3600  3700  3800  3900  4000  
CCTV/security  ...                                                              
0              ...  1288  1273  1238  1387  1454  1540  1427  1200  2284   875  
1              ...  1728  1737  1811  1951  2096  2118  1869  1638  3340  1128  

[2 rows x 41 columns]
CCTV/security  x  neighbourhood
Pval =  0.5580863256285917  Распределения одинаковы
neighbourhood     0   100   200   30

In [142]:
 two_way_inter_train.head(5)

,room x layout_type,room x property_type,room x furnish_type,room x bathroom,room x city,room x parking_spaces,room x floor,room x pet_friendly,room x power_backup,room x washing_machine,...,air_conditioner x geyser/solar,air_conditioner x CCTV/security,air_conditioner x lift,air_conditioner x neighbourhood,geyser/solar x CCTV/security,geyser/solar x lift,geyser/solar x neighbourhood,CCTV/security x lift,CCTV/security x neighbourhood,lift x neighbourhood
0,3 x BHK,3 x Independent House,3 x Furnished,3 x 2,3 x Chennai,3 x 1,3 x 1,3 x 1,3 x 0,3 x 0,...,1 x 0,1 x 0,1 x 0,1 x 300,0 x 0,0 x 0,0 x 300,0 x 0,0 x 300,0 x 300
1,1 x BHK,1 x Apartment,1 x Unfurnished,1 x 2,1 x Hyderabad,1 x 0,1 x 17,1 x 0,1 x 1,1 x 0,...,0 x 1,0 x 0,0 x 1,0 x 1600,1 x 0,1 x 1,1 x 1600,0 x 1,0 x 1600,1 x 1600
2,3 x BHK,3 x Apartment,3 x Semi-Furnished,3 x 2,3 x Kolkata,3 x 0,3 x 10,3 x 1,3 x 1,3 x 1,...,1 x 0,1 x 0,1 x 1,1 x 3100,0 x 0,0 x 1,0 x 3100,0 x 1,0 x 3100,1 x 3100
3,1 x BHK,1 x Apartment,1 x Unfurnished,1 x 1,1 x Mumbai,1 x 0,1 x 18,1 x 0,1 x 0,1 x 0,...,0 x 0,0 x 0,0 x 1,0 x 300,0 x 0,0 x 1,0 x 300,0 x 1,0 x 300,1 x 300
4,1 x BHK,1 x Studio Apartment,1 x Semi-Furnished,1 x 3,1 x Pune,1 x 0,1 x 14,1 x 0,1 x 0,1 x 1,...,1 x 0,1 x 0,1 x 1,1 x 0,0 x 0,0 x 1,0 x 0,0 x 1,0 x 0,1 x 0


In [143]:
 two_way_inter_test.head(5)

,room x layout_type,room x property_type,room x furnish_type,room x bathroom,room x city,room x parking_spaces,room x floor,room x pet_friendly,room x power_backup,room x washing_machine,...,air_conditioner x geyser/solar,air_conditioner x CCTV/security,air_conditioner x lift,air_conditioner x neighbourhood,geyser/solar x CCTV/security,geyser/solar x lift,geyser/solar x neighbourhood,CCTV/security x lift,CCTV/security x neighbourhood,lift x neighbourhood
0,2 x BHK,2 x Independent Floor,2 x Semi-Furnished,2 x 1,2 x Mumbai,2 x 0,2 x 2,2 x 0,2 x 1,2 x 1,...,1 x 0,1 x 1,1 x 0,1 x 900,0 x 1,0 x 0,0 x 900,1 x 0,1 x 900,0 x 900
1,1 x BHK,1 x Independent House,1 x Semi-Furnished,1 x 3,1 x Hyderabad,1 x 1,1 x 4,1 x 0,1 x 0,1 x 1,...,1 x 0,1 x 0,1 x 0,1 x 2500,0 x 0,0 x 0,0 x 2500,0 x 0,0 x 2500,0 x 2500
2,1 x BHK,1 x Apartment,1 x Unfurnished,1 x 1,1 x Hyderabad,1 x 1,1 x 3,1 x 1,1 x 1,1 x 0,...,0 x 0,0 x 0,0 x 0,0 x 3200,0 x 0,0 x 0,0 x 3200,0 x 0,0 x 3200,0 x 3200
3,3 x BHK,3 x Apartment,3 x Unfurnished,3 x 5,3 x Mumbai,3 x 1,3 x 9,3 x 1,3 x 0,3 x 0,...,0 x 1,0 x 1,0 x 1,0 x 1200,1 x 1,1 x 1,1 x 1200,1 x 1,1 x 1200,1 x 1200
4,2 x BHK,2 x Apartment,2 x Semi-Furnished,2 x 2,2 x Bangalore,2 x 1,2 x 17,2 x 0,2 x 1,2 x 1,...,1 x 0,1 x 0,1 x 1,1 x 1000,0 x 0,0 x 1,0 x 1000,0 x 1,0 x 1000,1 x 1000


Сохранение датафреймов с парными взаимодействиями.

In [144]:
two_way_inter_train.to_csv('two_way_inter_train.csv' , sep=',' , index = False)
two_way_inter_test.to_csv('two_way_inter_test.csv' , sep=',' , index = False)

Проверка тройных взаимодействий категориальных переменных.

Проверка равенства распределений в train и test.

Отбираются комбинации признаков с минимальном количеством объектов в каждом сочетании, равным 21.

In [14]:
threeway_inter_cols  = [] 
for i in range( len(cat_cols )-2):
    for k in range( i+1,  len(cat_cols )-1 ):
        for w in range( k+1,  len(cat_cols )):
            print('========================================================') 
            s = pd.crosstab( train[ cat_cols[i] ] , columns =[ train[ cat_cols[k] ], train[ cat_cols[w] ]] ).values   
             
            if ( s[s<=20].size == 0 ):
                print(cat_cols[i] , ' x ', cat_cols[k], ' x ', cat_cols[w])
                trn_inter = train[cat_cols[i] ].astype('str')+' x ' + train[ cat_cols[k]].astype('str')+' x ' + train[ cat_cols[w]].astype('str')
                tst_inter = test[cat_cols[i] ].astype('str')+' x ' + test[cat_cols[k]].astype('str')+' x ' + test[cat_cols[w]].astype('str')
         
                stat , pv =  kstest(rvs = trn_inter , 
                        cdf = tst_inter)
                if pv < 0.05:
                    print('Pval = ',pv  ,' Распределения неодинаковы')
                else:
                    threeway_inter_cols.append(cat_cols[i] + ' x '+ cat_cols[k]+ ' x '+ cat_cols[w]   ) 
                    print('Pval = ',pv  ,' Распределения одинаковы')
                
 

room  x  layout_type  x  furnish_type
Pval =  0.9732287557428543  Распределения одинаковы
room  x  layout_type  x  parking_spaces
Pval =  0.9092541649323382  Распределения одинаковы
room  x  layout_type  x  pet_friendly
Pval =  0.9732287557428543  Распределения одинаковы
room  x  layout_type  x  power_backup
Pval =  0.9732287557428543  Распределения одинаковы
room  x  layout_type  x  washing_machine
Pval =  0.9732287557428543  Распределения одинаковы
room  x  layout_type  x  air_conditioner
Pval =  0.9664790521597875  Распределения одинаковы
room  x  layout_type  x  geyser/solar
Pval =  0.9732287557428543  Распределения одинаковы
room  x  layout_type  x  CCTV/security
Pval =  0.9732287557428543  Распределения одинаковы
room  x  layout_type  x  lift
Pval =  0.9732287557428543  Распределения одинаковы
room  x  furnish_type  x  parking_spaces
Pval =  0.8915195786785064  Распределения одинаковы
room  x  furnish_type  x  pet_friendly
Pval =  0.9635834043607904  Распределения одинаковы
room 

Pval =  0.9240407502411938  Распределения одинаковы
room  x  washing_machine  x  air_conditioner
Pval =  0.9732287557428543  Распределения одинаковы
room  x  washing_machine  x  geyser/solar
Pval =  0.5206273933971802  Распределения одинаковы
room  x  washing_machine  x  CCTV/security
Pval =  0.9732287557428543  Распределения одинаковы
room  x  washing_machine  x  lift
Pval =  0.9732287557428543  Распределения одинаковы
room  x  air_conditioner  x  geyser/solar
Pval =  0.6995433007329466  Распределения одинаковы
room  x  air_conditioner  x  CCTV/security
Pval =  0.9732287557428543  Распределения одинаковы
room  x  air_conditioner  x  lift
Pval =  0.9732287557428543  Распределения одинаковы
room  x  geyser/solar  x  CCTV/security
Pval =  0.9210254414062311  Распределения одинаковы
room  x  geyser/solar  x  lift
Pval =  0.9291016454889924  Распределения одинаковы
room  x  CCTV/security  x  lift
Pval =  0.9732287557428543  Распределения одинаковы
layout_type  x  property_type  x  parking_

Pval =  0.7035022318647677  Распределения одинаковы
layout_type  x  parking_spaces  x  washing_machine
Pval =  0.7035022318647677  Распределения одинаковы
layout_type  x  parking_spaces  x  air_conditioner
Pval =  0.6460421908864331  Распределения одинаковы
layout_type  x  parking_spaces  x  geyser/solar
Pval =  0.7035022318647677  Распределения одинаковы
layout_type  x  parking_spaces  x  CCTV/security
Pval =  0.5820957779349832  Распределения одинаковы
layout_type  x  parking_spaces  x  lift
Pval =  0.7035022318647677  Распределения одинаковы
layout_type  x  parking_spaces  x  neighbourhood
Pval =  0.17536798970619627  Распределения одинаковы
layout_type  x  floor  x  pet_friendly
Pval =  0.5339566021430872  Распределения одинаковы
layout_type  x  floor  x  power_backup
Pval =  0.6173219419468263  Распределения одинаковы
layout_type  x  floor  x  washing_machine
Pval =  0.6214734903488702  Распределения одинаковы
layout_type  x  floor  x  air_conditioner
Pval =  0.6239790477260172  Р

property_type  x  parking_spaces  x  pet_friendly
Pval =  0.9778982195512751  Распределения одинаковы
property_type  x  parking_spaces  x  power_backup
Pval =  0.8726092339353183  Распределения одинаковы
property_type  x  parking_spaces  x  washing_machine
Pval =  0.9778982195512751  Распределения одинаковы
property_type  x  parking_spaces  x  air_conditioner
Pval =  0.9403612312826695  Распределения одинаковы
property_type  x  parking_spaces  x  geyser/solar
Pval =  0.9778982195512751  Распределения одинаковы
property_type  x  parking_spaces  x  CCTV/security
Pval =  0.7027403438303588  Распределения одинаковы
property_type  x  parking_spaces  x  lift
Pval =  0.9394867554375025  Распределения одинаковы
property_type  x  pet_friendly  x  power_backup
Pval =  0.9528138401243438  Распределения одинаковы
property_type  x  pet_friendly  x  washing_machine
Pval =  0.833907272398756  Распределения одинаковы
property_type  x  pet_friendly  x  air_conditioner
Pval =  0.4913398814077954  Распре

Pval =  0.9056882591179618  Распределения одинаковы
furnish_type  x  parking_spaces  x  floor
Pval =  0.9999829310473873  Распределения одинаковы
furnish_type  x  parking_spaces  x  pet_friendly
Pval =  0.9996160483023867  Распределения одинаковы
furnish_type  x  parking_spaces  x  power_backup
Pval =  0.9985869294088304  Распределения одинаковы
furnish_type  x  parking_spaces  x  geyser/solar
Pval =  0.9996023688592778  Распределения одинаковы
furnish_type  x  parking_spaces  x  CCTV/security
Pval =  0.9988435140497546  Распределения одинаковы
furnish_type  x  parking_spaces  x  lift
Pval =  0.9999999916859272  Распределения одинаковы
furnish_type  x  parking_spaces  x  neighbourhood
Pval =  0.9234581722018163  Распределения одинаковы
furnish_type  x  floor  x  pet_friendly
Pval =  0.9888912252846516  Распределения одинаковы
furnish_type  x  floor  x  power_backup
Pval =  0.9981948098732727  Распределения одинаковы
furnish_type  x  floor  x  geyser/solar
Pval =  0.9981948098732727  Ра

Pval =  0.8357169079464706  Распределения одинаковы
bathroom  x  floor  x  CCTV/security
Pval =  0.8476062774815278  Распределения одинаковы
bathroom  x  floor  x  lift
Pval =  0.867770491105198  Распределения одинаковы
bathroom  x  pet_friendly  x  power_backup
Pval =  0.9992048931418311  Распределения одинаковы
bathroom  x  pet_friendly  x  washing_machine
Pval =  0.8421878169152277  Распределения одинаковы
bathroom  x  pet_friendly  x  air_conditioner
Pval =  0.9992048931418311  Распределения одинаковы
bathroom  x  pet_friendly  x  geyser/solar
Pval =  0.8506723164156593  Распределения одинаковы
bathroom  x  pet_friendly  x  CCTV/security
Pval =  0.9992048931418311  Распределения одинаковы
bathroom  x  pet_friendly  x  lift
Pval =  0.9992048931418311  Распределения одинаковы
bathroom  x  power_backup  x  washing_machine
Pval =  0.9931884109723282  Распределения одинаковы
bathroom  x  power_backup  x  air_conditioner
Pval =  0.9806220929852086  Распределения одинаковы
bathroom  x  po

Pval =  0.07671721274175658  Распределения одинаковы
city  x  washing_machine  x  air_conditioner
Pval =  0.09151488609445735  Распределения одинаковы
city  x  washing_machine  x  geyser/solar
Pval =  0.08513909781251106  Распределения одинаковы
city  x  washing_machine  x  CCTV/security
Pval =  0.08642659177493539  Распределения одинаковы
city  x  washing_machine  x  lift
Pval =  0.09094356860537567  Распределения одинаковы
city  x  washing_machine  x  neighbourhood
Pval =  0.07820350857256086  Распределения одинаковы
city  x  air_conditioner  x  geyser/solar
Pval =  0.06279587299174005  Распределения одинаковы
city  x  air_conditioner  x  CCTV/security
Pval =  0.09394392819717734  Распределения одинаковы
city  x  air_conditioner  x  lift
Pval =  0.1216577113876699  Распределения одинаковы
city  x  geyser/solar  x  CCTV/security
Pval =  0.032052781535795494  Распределения неодинаковы
city  x  geyser/solar  x  lift
Pval =  0.10697241698856408  Распределения одинаковы
city  x  geyser/so

Pval =  0.9154231101891309  Распределения одинаковы
floor  x  pet_friendly  x  geyser/solar
Pval =  0.9154231101891309  Распределения одинаковы
floor  x  pet_friendly  x  CCTV/security
Pval =  0.9154231101891309  Распределения одинаковы
floor  x  pet_friendly  x  neighbourhood
Pval =  0.7745085164450699  Распределения одинаковы
floor  x  power_backup  x  washing_machine
Pval =  0.6117185993211673  Распределения одинаковы
floor  x  power_backup  x  air_conditioner
Pval =  0.6763566613825955  Распределения одинаковы
floor  x  power_backup  x  geyser/solar
Pval =  0.6763566613825955  Распределения одинаковы
floor  x  power_backup  x  CCTV/security
Pval =  0.6763566613825955  Распределения одинаковы
floor  x  washing_machine  x  air_conditioner
Pval =  0.838998444787209  Распределения одинаковы
floor  x  washing_machine  x  geyser/solar
Pval =  0.6435429576904115  Распределения одинаковы
floor  x  washing_machine  x  CCTV/security
Pval =  0.838998444787209  Распределения одинаковы
floor  x

Pval =  0.39536359605780835  Распределения одинаковы
power_backup  x  CCTV/security  x  neighbourhood
Pval =  0.32152535017962947  Распределения одинаковы
power_backup  x  lift  x  neighbourhood
Pval =  0.08011994927414412  Распределения одинаковы
washing_machine  x  geyser/solar  x  CCTV/security
Pval =  0.4263626451456167  Распределения одинаковы
washing_machine  x  geyser/solar  x  lift
Pval =  0.7165570212250356  Распределения одинаковы
washing_machine  x  geyser/solar  x  neighbourhood
Pval =  0.6118125331752218  Распределения одинаковы
washing_machine  x  CCTV/security  x  lift
Pval =  0.9999999771003735  Распределения одинаковы
washing_machine  x  CCTV/security  x  neighbourhood
Pval =  0.9891323496451175  Распределения одинаковы
washing_machine  x  lift  x  neighbourhood
Pval =  0.9077874647513601  Распределения одинаковы
air_conditioner  x  geyser/solar  x  CCTV/security
Pval =  0.7996926622802354  Распределения одинаковы
air_conditioner  x  geyser/solar  x  lift
Pval =  0.820

In [22]:
len(threeway_inter_cols)

392

In [25]:
three_way_inter_train = pd.DataFrame()
three_way_inter_test = pd.DataFrame()

for I in tqdm( threeway_inter_cols ):
    W = I.split(' x ')
    trn_inter = train[ W[0] ].astype('str')+' x ' + train[W[1] ].astype('str')+' x ' + train[ W[2] ].astype('str')
    tst_inter = test[W[0]  ].astype('str')+' x ' + test[W[1] ].astype('str')+' x ' + test[W[2] ].astype('str')  

    three_way_inter_train = pd.concat([three_way_inter_train ,trn_inter ] , axis=1)
    three_way_inter_test = pd.concat([three_way_inter_test ,tst_inter ] , axis=1)

three_way_inter_train.columns = threeway_inter_cols
three_way_inter_test.columns =  threeway_inter_cols

100%|████████████████████████████████████████| 392/392 [21:43<00:00,  3.33s/it]


In [26]:
three_way_inter_train.head()

,room x layout_type x furnish_type,room x layout_type x parking_spaces,room x layout_type x pet_friendly,room x layout_type x power_backup,room x layout_type x washing_machine,room x layout_type x air_conditioner,room x layout_type x geyser/solar,room x layout_type x CCTV/security,room x layout_type x lift,room x furnish_type x parking_spaces,room x furnish_type x pet_friendly,room x furnish_type x power_backup,room x furnish_type x washing_machine,room x furnish_type x air_conditioner,room x furnish_type x geyser/solar,room x furnish_type x CCTV/security,room x furnish_type x lift,room x parking_spaces x pet_friendly,room x parking_spaces x power_backup,room x parking_spaces x washing_machine,room x parking_spaces x air_conditioner,room x parking_spaces x geyser/solar,room x parking_spaces x CCTV/security,room x parking_spaces x lift,room x floor x lift,room x pet_friendly x power_backup,room x pet_friendly x washing_machine,room x pet_friendly x air_conditioner,room x pet_friendly x geyser/solar,room x pet_friendly x CCTV/security,room x pet_friendly x lift,room x power_backup x washing_machine,room x power_backup x air_conditioner,room x power_backup x geyser/solar,room x power_backup x CCTV/security,room x power_backup x lift,room x washing_machine x air_conditioner,room x washing_machine x geyser/solar,room x washing_machine x CCTV/security,room x washing_machine x lift,room x air_conditioner x geyser/solar,room x air_conditioner x CCTV/security,room x air_conditioner x lift,room x geyser/solar x CCTV/security,room x geyser/solar x lift,room x CCTV/security x lift,layout_type x property_type x parking_spaces,layout_type x property_type x pet_friendly,layout_type x property_type x power_backup,layout_type x property_type x air_conditioner,...,pet_friendly x power_backup x geyser/solar,pet_friendly x power_backup x CCTV/security,pet_friendly x power_backup x lift,pet_friendly x power_backup x neighbourhood,pet_friendly x washing_machine x air_conditioner,pet_friendly x washing_machine x geyser/solar,pet_friendly x washing_machine x CCTV/security,pet_friendly x washing_machine x lift,pet_friendly x washing_machine x neighbourhood,pet_friendly x air_conditioner x geyser/solar,pet_friendly x air_conditioner x CCTV/security,pet_friendly x air_conditioner x lift,pet_friendly x air_conditioner x neighbourhood,pet_friendly x geyser/solar x CCTV/security,pet_friendly x geyser/solar x lift,pet_friendly x geyser/solar x neighbourhood,pet_friendly x CCTV/security x lift,pet_friendly x CCTV/security x neighbourhood,pet_friendly x lift x neighbourhood,power_backup x washing_machine x air_conditioner,power_backup x washing_machine x geyser/solar,power_backup x washing_machine x CCTV/security,power_backup x washing_machine x lift,power_backup x washing_machine x neighbourhood,power_backup x air_conditioner x geyser/solar,power_backup x air_conditioner x CCTV/security,power_backup x air_conditioner x lift,power_backup x air_conditioner x neighbourhood,power_backup x geyser/solar x CCTV/security,power_backup x geyser/solar x lift,power_backup x geyser/solar x neighbourhood,power_backup x CCTV/security x lift,power_backup x CCTV/security x neighbourhood,power_backup x lift x neighbourhood,washing_machine x geyser/solar x CCTV/security,washing_machine x geyser/solar x lift,washing_machine x geyser/solar x neighbourhood,washing_machine x CCTV/security x lift,washing_machine x CCTV/security x neighbourhood,washing_machine x lift x neighbourhood,air_conditioner x geyser/solar x CCTV/security,air_conditioner x geyser/solar x lift,air_conditioner x geyser/solar x neighbourhood,air_conditioner x CCTV/security x lift,air_conditioner x CCTV/security x neighbourhood,air_conditioner x lift x neighbourhood,geyser/solar x CCTV/security x lift,geyser/solar x CCTV/security x neighbourhood,geyser/solar x lift x neighbourhood,CCTV/security x lift x neighbourhood
0,3 x BHK x Furnished,3 x BHK x 1,3 x BHK x 1,3 x BHK x 0,3 x BHK x 0,3 x BHK x 1,3 x BHK x 0,3 x

In [27]:
three_way_inter_test.head()

,room x layout_type x furnish_type,room x layout_type x parking_spaces,room x layout_type x pet_friendly,room x layout_type x power_backup,room x layout_type x washing_machine,room x layout_type x air_conditioner,room x layout_type x geyser/solar,room x layout_type x CCTV/security,room x layout_type x lift,room x furnish_type x parking_spaces,room x furnish_type x pet_friendly,room x furnish_type x power_backup,room x furnish_type x washing_machine,room x furnish_type x air_conditioner,room x furnish_type x geyser/solar,room x furnish_type x CCTV/security,room x furnish_type x lift,room x parking_spaces x pet_friendly,room x parking_spaces x power_backup,room x parking_spaces x washing_machine,room x parking_spaces x air_conditioner,room x parking_spaces x geyser/solar,room x parking_spaces x CCTV/security,room x parking_spaces x lift,room x floor x lift,room x pet_friendly x power_backup,room x pet_friendly x washing_machine,room x pet_friendly x air_conditioner,room x pet_friendly x geyser/solar,room x pet_friendly x CCTV/security,room x pet_friendly x lift,room x power_backup x washing_machine,room x power_backup x air_conditioner,room x power_backup x geyser/solar,room x power_backup x CCTV/security,room x power_backup x lift,room x washing_machine x air_conditioner,room x washing_machine x geyser/solar,room x washing_machine x CCTV/security,room x washing_machine x lift,room x air_conditioner x geyser/solar,room x air_conditioner x CCTV/security,room x air_conditioner x lift,room x geyser/solar x CCTV/security,room x geyser/solar x lift,room x CCTV/security x lift,layout_type x property_type x parking_spaces,layout_type x property_type x pet_friendly,layout_type x property_type x power_backup,layout_type x property_type x air_conditioner,...,pet_friendly x power_backup x geyser/solar,pet_friendly x power_backup x CCTV/security,pet_friendly x power_backup x lift,pet_friendly x power_backup x neighbourhood,pet_friendly x washing_machine x air_conditioner,pet_friendly x washing_machine x geyser/solar,pet_friendly x washing_machine x CCTV/security,pet_friendly x washing_machine x lift,pet_friendly x washing_machine x neighbourhood,pet_friendly x air_conditioner x geyser/solar,pet_friendly x air_conditioner x CCTV/security,pet_friendly x air_conditioner x lift,pet_friendly x air_conditioner x neighbourhood,pet_friendly x geyser/solar x CCTV/security,pet_friendly x geyser/solar x lift,pet_friendly x geyser/solar x neighbourhood,pet_friendly x CCTV/security x lift,pet_friendly x CCTV/security x neighbourhood,pet_friendly x lift x neighbourhood,power_backup x washing_machine x air_conditioner,power_backup x washing_machine x geyser/solar,power_backup x washing_machine x CCTV/security,power_backup x washing_machine x lift,power_backup x washing_machine x neighbourhood,power_backup x air_conditioner x geyser/solar,power_backup x air_conditioner x CCTV/security,power_backup x air_conditioner x lift,power_backup x air_conditioner x neighbourhood,power_backup x geyser/solar x CCTV/security,power_backup x geyser/solar x lift,power_backup x geyser/solar x neighbourhood,power_backup x CCTV/security x lift,power_backup x CCTV/security x neighbourhood,power_backup x lift x neighbourhood,washing_machine x geyser/solar x CCTV/security,washing_machine x geyser/solar x lift,washing_machine x geyser/solar x neighbourhood,washing_machine x CCTV/security x lift,washing_machine x CCTV/security x neighbourhood,washing_machine x lift x neighbourhood,air_conditioner x geyser/solar x CCTV/security,air_conditioner x geyser/solar x lift,air_conditioner x geyser/solar x neighbourhood,air_conditioner x CCTV/security x lift,air_conditioner x CCTV/security x neighbourhood,air_conditioner x lift x neighbourhood,geyser/solar x CCTV/security x lift,geyser/solar x CCTV/security x neighbourhood,geyser/solar x lift x neighbourhood,CCTV/security x lift x neighbourhood
0,2 x BHK x Semi-Furnished,2 x BHK x 0,2 x BHK x 0,2 x BHK x 1,2 x BHK x 1,2 x BHK x 1,2 x BHK x 

Сохранение датафреймов с тройными взаимодействиями.

In [28]:
three_way_inter_train.to_csv('three_way_inter_train.csv' , sep=',' , index = False)
three_way_inter_test.to_csv('three_way_inter_test.csv' , sep=',' , index = False)